<a href="https://colab.research.google.com/github/saurabhlalsaxena/LinkedIn-Post-Creator/blob/main/LinkedIn_Post_Creator_Streamlit_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Setup

In [1]:
!pip install -q linkedin-api-client

In [2]:
!pip install -q -U langchain-cli langchain langchain_core langgraph langchain_community

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.4/116.4 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.3/207.3 kB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.5/59.5 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.6/94.6 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.

In [3]:
!pip install -q openai langchain_openai langchain-anthropic langchain-mistralai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.5/383.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 894.0/894.0 kB 14.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 16.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 33.9 MB/s eta 0:00:00


In [4]:
!pip install -q duckduckgo-search

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 25.5 MB/s eta 0:00:00


In [5]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 52.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 4.0 MB/s eta 0:00:00


In [6]:
import os
from google.colab import userdata

os.environ["ACCESS_TOKEN"] = userdata.get('LINKEDIN_ACCESS_TOKEN')
os.environ["ANTHROPIC_API_KEY"] = userdata.get('ANTHROPIC_API_KEY')

#Streamlit Application

## Create App

In [10]:
%%writefile linkedInPostCreator.py
import streamlit as st
from linkedin_api.clients.restli.client import RestliClient
from duckduckgo_search import AsyncDDGS
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain_mistralai.chat_models import ChatMistralAI
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import BaseMessage, HumanMessage
import asyncio
import datetime
import json
import os
import time
import re

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")

#LinkedIn Utils
restli_client = RestliClient()

def get_person_urn(ACCESS_TOKEN):
  response = restli_client.get(
      resource_path="/userinfo",
      access_token=ACCESS_TOKEN
    )
  return response.entity['sub']

def create_text_post(urn, content,ACCESS_TOKEN):
  response = restli_client.create(
        resource_path="/posts",
        entity={
            "author": f"urn:li:person:{urn}",
            "lifecycleState": "PUBLISHED",
            "visibility": "PUBLIC",
            "commentary": content,
            "distribution": {
                "feedDistribution": "MAIN_FEED",
                "targetEntities": [],
                "thirdPartyDistributionChannels": [],
            },
        },
        access_token=ACCESS_TOKEN,
    )
  status = response.status_code
  post_id = response.entity_id
  return status, post_id

# duckduckgo utils
async def aget_results(word):
    """
    Asynchronously fetch search results from DuckDuckGo for a given word.

    Args:
        word (str): The search query.

    Returns:
        list: A list of search results (max 4) for the given query.
    """
    results = await AsyncDDGS(proxy=None).atext(word, max_results=4)
    return results

async def search(queries):
    """
    Perform asynchronous searches for multiple queries.

    Args:
        queries (list): A list of search queries.

    Returns:
        list: A list of search results for each query.
    """
    words = queries
    tasks = [aget_results(w) for w in words]
    results = await asyncio.gather(*tasks)
    return results


#ReACT Agent
@tool
def search_tool(query):
    """
    A search tool that finds up-to-date information on a given topic.

    This function takes a search query, combines it with today's date, and returns
    relevant search results. It's useful for finding current information on any topic.

    Args:
        query (str): The search query string. This should be the main topic or
                     keywords for the search operation.

    Returns:
        list: A list of search results related to the query and today's date.
    """
    today = datetime.datetime.now().strftime("%d %B %Y")
    word = f"{query}, date : {today}"
    results = asyncio.run(aget_results(word))
    return results

llm = ChatAnthropic(temperature=0, model_name="claude-3-5-sonnet-20240620", max_tokens=4096)
#llm = ChatMistralAI(api_key=os.environ.get('MISTRAL_API_KEY'),model="mistral-large-latest",temperature=0)
#llm = ChatOpenAI(model="gpt-4o-mini",temperature=0, streaming=True)

# setup the toolkit
tools = [search_tool]

# Create a react agent
post_writer = create_react_agent(llm, tools=tools)

def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()
    return message

system_prompt = """You are a LinkedIn post writing Assistant.
Given a topic or post concept, create a LinkedIn post in the tone specified in less than 150 words.
You also have access to a internet search tool to help you find up-to-date information on a given topic.
Mention the sources for your data. Ensure you add the URL for the data source.
IMPORTANT:
Place the entire LinkedIn post between <LINKEDIN POST> and </LINKEDIN POST> tags.
"""

def escape_text(text):
    chars = ["\\", "|", "{", "}", "@", "[", "]", "(", ")", "<", ">", "#", "*", "_", "~"]
    for char in chars:
        text = text.replace(char, "\\"+char)
    return text

def post(linkedin_post):
    content = escape_text(linkedin_post)
    try:
        print("Getting URN")
        urn = get_person_urn(ACCESS_TOKEN)
        status, post_id = create_text_post(urn, content, ACCESS_TOKEN)
        print(f"The Post Status: {status} and Post ID: {post_id}")
        return status, post_id
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        raise  # Re-raise the exception to be caught in the main function

# Main Streamlit app
def main():
    st.title("LinkedIn Post Creator")

    # Initialize session state
    if 'edited_output' not in st.session_state:
        st.session_state.edited_output = ""
    if 'post_status' not in st.session_state:
        st.session_state.status = None
    if 'post_id' not in st.session_state:
        st.session_state.post_id = None
    if 'show_post_button' not in st.session_state:
        st.session_state.show_post_button = False

    # Text input for post description
    post_description = st.text_input("Post Description")

    # Dropdown for post tone
    post_tone = st.selectbox("Post Tone", ["Enthusiastic", "Thoughtful", "Educational", "Assertive", "Professional", "Funny"])

    # Create button
    if st.button("Create"):
        user_prompt = f"Topic/Post Concept : <topic> {post_description} </topic> Tone : <tone> {post_tone} </tone>"
        inputs = {"messages": [("system", system_prompt),("user", user_prompt)]}
        config = {"recursion_limit": 100}

        # Print the stream and get the final output
        final_output = print_stream(post_writer.stream(inputs,config, stream_mode="values"))

        # Display the final output in an editable text area
        st.subheader("Generated Post")
        linkedin_post_match = re.search(r'<LINKEDIN POST>(.*?)</LINKEDIN POST>', final_output.content, re.DOTALL)
        linkedin_post = linkedin_post_match.group(1).strip()
        st.session_state.edited_output = linkedin_post
        st.session_state.show_post_button = True

    # Text area for editing (only shown after Create button is pressed)
    if st.session_state.show_post_button:
        st.session_state.edited_output = st.text_area("Edit your post here:", value=st.session_state.edited_output, height=500, key="editable_text_area")

    # Post button (only shown after Create button is pressed)
    if st.session_state.show_post_button is True:
        if st.button("Post"):
            if st.session_state.edited_output:
                st.session_state.status, st.session_state.post_id = post(st.session_state.edited_output)
                if st.session_state.status and st.session_state.post_id:
                    st.write(f"The Post Status: {st.session_state.status} and Post ID: {st.session_state.post_id}")
                    st.session_state.show_post_button = False

if __name__ == "__main__":
    main()

Overwriting linkedInPostCreator.py


## Run App

In [11]:
# workaround
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!ls

linkedInPostCreator.py	sample_data


In [12]:
!streamlit run linkedInPostCreator.py & npx localtunnel --port 8501




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.80.199.165:8501

your url is: https://silver-moments-beg.loca.lt
  Stopping...
^C
